**Pré - Tutorial**

In [ ]:
!pip install yfinance
!pip install fpdf
!pip install --upgrade matplotlib

In [ ]:
def grafico_media_movel(input_data, ticker,window = 10,file_name = 'SMA_EMA_plot.png'):

  #Gerar SMA e EMA
  input_data[f'SMA{window}'] = input_data['Adj Close'].rolling(window).mean()
  input_data[f'EMA{window}'] = input_data['Adj Close'].ewm(span=window).mean()

  #Criar Figura e Grafico
  fig,ax = plt.subplots()

  #Adicionar Serie Temporal
  ax.plot(input_data['Date'],input_data['Adj Close'],label=ticker)
  ax.plot(input_data['Date'],input_data['SMA10'],label='SMA10')
  ax.plot(input_data['Date'],input_data['EMA10'],label='EMA10')

  #Formatar para aparecer o simbolo do dolar
  ax.yaxis.set_major_formatter('${x:}') #so funciona com a versão mais recente do matplotlib

  #ax.set_ylabel('Adj Close')
  ax.set_title('Média Móvel Simples e Exponencial') #Adicionar titulo
  ax.legend(loc='lower right') #apresentar labels
  #ax.xaxis_date() 
  fig.patch.set_visible(False) #esconder cinzento
  fig.autofmt_xdate() #rodar eixo do X

  #Gravar Figura
  fig.savefig(file_name,bbox_inches='tight',pad_inches = 0.1,dpi=300) #500

  print(f'Grafico {file_name} criado!')

_

**Tutorial**

In [ ]:
from fpdf import FPDF
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
#input

tickers = 'TSLA' #'AAPL'

#dates
start = '2020-05-01'
end = '2020-12-01'

In [ ]:
appl = yf.Ticker(tickers)
#appl.info

keys_to_extract = ['market','longName','industry','marketCap'
,'priceToBook','website','symbol','shortName','averageDailyVolume10Day','payoutRatio','longBusinessSummary']

a_subset = {key: appl.info[key] for key in keys_to_extract}
a_subset

{'averageDailyVolume10Day': 51960957,
 'industry': 'Auto Manufacturers',
 'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, Netherlands, Norway, and internationally. The company operates in two segments, Automotive; and Energy Generation and Storage. The Automotive segment offers sedans and sport utility vehicles. It also provides electric powertrain components and systems; and services for electric vehicles through its company-owned service locations, and Tesla mobile service technicians, as well as sells used vehicles. This segment markets and sells its products through a network of company-owned stores and galleries, as well as through its own Website. The Energy Generation and Storage segment offers energy storage products, such as rechargeable lithium-ion battery systems for use in homes, industrial, commercial facilities, and utility grids; and designs, m

In [ ]:
input_data = yf.download(tickers,start=start,end=end).reset_index()
input_data.tail()

In [ ]:
grafico_media_movel(input_data,tickers,10,file_name='SMA_EMA_plot.png')

Relatorio PDF

In [ ]:
#Set Variables
largura = 210 #mm
altura = 297 #mm
margem = 10 #mm

######### Set Configuration #########
pdf = FPDF('P','mm','A4')
pdf.add_page()
pdf.set_font('arial','B',18)
pdf.set_auto_page_break(auto=bool,margin=margem)

##### Colocar Cabeçalho e Rodapé ################
pdf.image('Letter_Page_Header.jpg',x=0,y=0,w=largura)
pdf.image('Letter_Page_Footer.jpg',x=0,y=altura-10,w=largura)

########## Inserir Titulo ###################
company = a_subset['shortName']
pdf.cell(largura-2*margem,60,f'Análise {company}',border=1,ln=1,align='R')
pdf.cell(60,5,'',1,1)

############### Secção Detalhes da Empresa ##############
pdf.set_font('arial','BU',12)
pdf.cell(60,10,'Detalhes da Empresa',1,1)

#Linha 1
add_row('Descrição:',a_subset['longName'],'Símbolo:',a_subset['symbol'])
#Linha 2
add_row('Mercado:',a_subset['market'],'Indústria:',a_subset['industry'])
pdf.cell(60,5,'',1,1)

########### Secção Indicadores ######################
pdf.set_font('arial','BU',12)
pdf.cell(60,10,'Indicadores',0,1)

#Linha 3
number = a_subset['marketCap']
market_cap = format(number,',').replace(",",".") + ' $'

add_row('Cap. Mercado:',market_cap,'PriceToBook',str(round(a_subset['priceToBook'],2)))

#Linha 4
number = a_subset['averageDailyVolume10Day']
avg_volume = format(number,',').replace(",",".")

add_row('Volume Diário:',avg_volume,'PayoutRatio:',str(round(a_subset['payoutRatio'],2)))
pdf.cell(60,5,'',0,1)

############ Grafico #################################
pdf.set_font('arial','BU',12)
pdf.cell(60,10,'Análise Gráfica',0,1)
img_path = 'SMA_EMA_plot.png'

pdf.image(img_path,x=pdf.get_x(),w=largura-2*margem,h=(altura-pdf.get_y()-2*margem))


### Rodapé #########################
pdf.set_font('arial','',8)
now = datetime.now().strftime("%d-%m-%Y")
nota_rodape = f'Análise {company}. Relatório gerado a {now}.'

pdf.text(x=largura - len(nota_rodape)*1.5 - margem,y=altura - margem/2,txt=nota_rodape)

pdf.output('Relatorio_Analise.pdf','F')


In [ ]:
def add_row(bold_1,text_1,bold_2,text_2):

  pdf.set_font('arial','B',12)
  pdf.cell(30,10,bold_1,0,0)

  pdf.set_font('arial','',12)
  pdf.cell(65,10,text_1,0,0,align='C')

  pdf.set_font('arial','B',12)
  pdf.cell(30,10,bold_2,0,0)

  pdf.set_font('arial','',12)
  pdf.cell(65,10,text_2,0,1,align='C')


